In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab06.ipynb")

<a id='verytop'></a>

# Lab 6: Applications of Singular Value Decomposition

## Due Date: Thurs, Dec 4th 11:59 PM on Gradescope


### Detailed Submission Instructions Are Provided at the end of this Notebook


## Collaboration Policy

A key step in learning and retention is **creating solutions on your own.**   Below are examples of acceptable vs unacceptable use of resources and collaboration when doing lab assignments in CSCI 2820.


The following would be some **examples of cheating** when working on HW assignments in CSCI 2820.  Any of these constitute a **violation of the course's collaboration policy and will result in an F in the course and a trip to the honor council**.   


 - Consulting web pages that may have a solution to a given lab problem or one similar is cheating.  However, consulting notes from the class videos, and web pages that explain the material taught in class but do NOT show a solution to the lab problem in question are permissible to view.  Clearly, there's a fuzzy line here between a valid use of resources and cheating. To avoid this line, one should merely consult the course videos, the course textbooks, and references that contain syntax and/or formulas.
 - Copying a segment of code or math solution of three lines or more from another student from a printout, handwritten copy, or by looking at their computer screen 
 - Allowing another student to copy a segment of your code or math solution of three lines or more
 - Taking a copy of another student's work (or a solution found online) and then editing that copy
 - Reading someone else’s solution to a problem on the lab before writing your own.
 - Asking someone to write all or part of a program or solution for you.
 - Asking someone else for the code necessary to fix the error for you, other than for simple syntactical errors
 


On the other hand, the following are some **examples of things which would NOT usually be
considered to be cheating**:
 - Working on a lab problem on your own first and then discussing with a classmate a particular part in the problem solution where you are stuck.  After clarifying any questions you should then continue to write your solution independently.
 - Asking someone (or searching online) how a particular construct in the language works.
 - Asking someone (or searching online) how to formulate a particular construct in the language.
 - Asking someone for help in finding an error in your program.  
 - Asking someone why a particular construct does not work as you expected in a given program.
   

To test whether you are truly doing your own work and retaining what you've learned you should be able to easily reproduce from scratch and explain a lab solution that was your own when asked in office hours by a TA/Instructor or on a quiz/exam.   


If you have difficulty in formulating the general solution to a problem on your own, or
you have difficulty in translating that general solution into a program, it is advisable to see
your instructor or teaching assistant rather than another student as this situation can easily
lead to a, possibly inadvertent, cheating situation.

We are here to help!  Visit office Hours and/or post questions on Piazza!



## Grading
Grading is broken down into autograded answers and manually graded answers. 

For autograded answers, the results of your code are compared to provided and/or hidden tests.

For manually graded answers you must show and explain all steps.  Graders will evaluate how well you answered the question and/or fulfilled the requirements of the question.


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


In [ ]:
# import useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
%matplotlib inline


# Function needed to run in-notebook tests
import hashlib

def get_hash(num):
    """Helper function for assessing correctness"""
    return hashlib.md5(str(num).encode()).hexdigest()


def get_array_hash_normalized(arr):
    """
    Hash numpy array ignoring dtype differences (values only),
    treating -0.0 as 0.0 and canonicalizing NaNs.
    """
    arr = np.ascontiguousarray(arr, dtype=np.float64)

    # View as unsigned integers so we can manipulate bits
    view = arr.view(np.uint64)

    # Mask for the exponent+mantissa (everything except the sign bit)
    mant_exp_mask = (1 << 63) - 1

    # For entries that are exactly zero (pos or neg), clear sign bit
    zero_mask = (view & mant_exp_mask) == 0
    view[zero_mask] = 0

    # Canonicalize NaNs: all NaNs get the same payload
    nan_mask = np.isnan(arr)
    if nan_mask.any():
        arr[nan_mask] = np.float64(np.nan)

    return hashlib.md5(arr.tobytes() + str(arr.shape).encode()).hexdigest()

## Slicing NumPy Matrices

Consider the matrix $$X = \begin{bmatrix} 4 & 11 & 14 \\ 8 & 7 & -2 \end{bmatrix}$$



In [ ]:
import numpy as np

We can write this a Numpy matrix as follows:

In [ ]:
X = np.array([[4, 11, 14],[8, 7, -2]])

X

We can find the Transpose as follows:

In [ ]:
# Find the Transpose
X.T

We can slice/extract rows/columns as follows:

In [ ]:
# Extract a single row (e.g., the second row)
row_slice = X[1, :]
print(f"Second row: {row_slice}") # Output: [5 6 7 8]



In [ ]:
# Extract a single column (e.g., the third column)
col_slice = X[:, 2]
print(f"Third column: {col_slice}") # Output: [ 3  7 11 15]



In [ ]:
# Extract a sub-matrix (e.g., rows 0-1, columns 1-2)
sub_matrix = X[0:2, 1:3]
print(f"Sub-matrix: \n{sub_matrix}")
# Output:
# [[2 3]
#  [6 7]]



In [ ]:
# Extract the first 2 columns
X[:, 0:2]

## Singular Value Decomposition

Recall, if we have a data matrix $X$, we can decompose it into $U$, $\Sigma$ and $V^T$ such that $X = U \Sigma V^T$.

This decomposition is called the **Singular Value Decomposition**

<img src="img/svd.png" alt="SVD" style="width:700px;height:auto;">

We'll use  NumPy's linear algebra module and its built-in SVD function `np.linalg.svd` to calculate the SVD.

In NumPy, `np.linalg.svd` returns:

$U$ — shape (m, m)

$S$ — 1-D **array of singular values**, length k = min(m, n).  $S$ is a little different in `NumPy`. Since the only useful values in the diagonal matrix $S$ are the singular values on the diagonal axis, only those nonzero values are returned and they are stored in an array.

$V^T$ — shape (n, n)



In [ ]:
U, S, Vt = np.linalg.svd(X)

In [ ]:
print("Shape of U", U.shape)
print("Shape of S", S.shape)
print("Shape of Vt", Vt.shape)

To convert S into a full diagonal matrix matching the original matrix shape (m, n), you can use np.zeros and np.fill_diagonal.

In [ ]:
# np.diag makes a diagonal matrix from the vector S

Sd = np.zeros(X.shape)

np.fill_diagonal(Sd, S)

Sd


In [ ]:
# Check that multiplying these 3 together gets us back to our original matrix:

U@Sd@Vt

#np.allclose(X, U@Sd@Vt)

# SVD and Rank r Approximations

Recall from our video preview that given an SVD $A =U \Sigma V^T $  we can use it to approximate $A$ as follows:
$$
\begin{aligned}
A \approx A_1 & = \sigma_1{\mathbf u}_1{\mathbf v}_1^T  \\
A \approx A_2 & = \sigma_1{\mathbf u}_1{\mathbf v}_1^T+\sigma_2{\mathbf u}_2{\mathbf v}_2^T \\
\vdots \\
A = A_r & = \sigma_1{\mathbf u}_1{\mathbf v}_1^T + \sigma_2{\mathbf u}_2{\mathbf v}_2^T + \ldots + \sigma_r{\mathbf u}_r{\mathbf v}_r^T
\end{aligned}
$$


This is the basis of how image compression can work.  Rather than storing  one number for each pixel, we could instead just store the three sets of singular values, left singular vectors, and right singular vectors.  

In [ ]:

# Run this cell below to load in data of a picture of a dog

from matplotlib.image import imread
import matplotlib.pyplot as plt
import os
plt.rcParams['figure.figsize'] = [16, 8]


A = imread(os.path.join('..','lab06/data','dog.jpg'))
X = np.mean(A, -1); # Convert RGB to grayscale

img = plt.imshow(X)
img.set_cmap('gray')
plt.axis('off')
plt.show()

In [ ]:
# DON'T UNCOMMMENT - TAKES TOO LONG ON CSEL


#U, S, VT = np.linalg.svd(X,full_matrices=False)
#S = np.diag(S)

#j = 0
#for r in (5, 20, 100):
    # Construct approximate image
#    Xapprox = U[:,:r] @ S[0:r,:r] @ VT[:r,:]
#    plt.figure(j+1)
#    j += 1
#    img = plt.imshow(Xapprox)
#    img.set_cmap('gray')
#    plt.axis('off')
#    plt.title('r = ' + str(r))
#    plt.show()

Here's the output of the code above:

<img src="img/compressed.png" alt="SVD" style="width:200px;height:auto;">

## Demo:  

Play around with this demonstration of image compression [using the SVD](http://timbaumann.info/svd-image-compression-demo/).

# SVD and Least Squares 


Recall: Least-squares problems arise when we are confronted with an inconsistent linear system  
$A \mathbf{x} = \mathbf{b}$.  
Since there is no exact solution to this system, we instead find the vector $\hat{\mathbf{x}}$, the least-squares approximate solution, by solving  
$A \hat{\mathbf{x}} = \hat{\mathbf{b}}$  where $\hat{\mathbf{b}}$ is the orthogonal projection of $\mathbf{b}$ onto $\operatorname{col}(A)$.








<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 1a  ###


Consider the equation $A\,\mathbf{x} = \mathbf{b}$ where  
$A = \begin{bmatrix} 1 & 0 \\ 1 & 1 \\ 1 & 2 \end{bmatrix}, \quad 
\mathbf{b} = \begin{bmatrix} -1 \\ 3 \\ 6 \end{bmatrix}.$

Find a singular value decomposition for $A$ using NumPy.  (You will have to convert the S output from NumPy into a diagonal matrix of the appropriate size - see the example above for how to do this). 
                                   


In [ ]:

A = ...

b = ...

U, S, Vt = ...

#The output from NumPy gives S as an array - convert to a diagonal matrix and save that as Sd


Sd = ...

...



# Check that this decomposition equals A 
np.isclose(U@Sd@Vt, A)


In [ ]:
grader.check("q1a")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 1b  ###


i). What are the singular values of $A$?  Give your answer as a NumPy array.

ii).  What is $r$, the rank of $A$? 


In [ ]:

singular_val = ...

r = ...

In [ ]:
grader.check("q1b")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 1c  ###


 Using $r$, the rank of $A$ you found above, form the reduced singular value decomposition $U_{r}\,\Sigma_{r}\,V_{r}^{T}$ by constructing:  
   - the matrix $U_{r}$, consisting of the first $r$ columns of $U$;  
   - the matrix $V_{r}$, consisting of the first $r$ columns of $V$;  
   - $\Sigma_r$, a square $r \times r$ diagonal matrix.  
   Verify that $A = U_{r}\,\Sigma_{r}\,V_{r}^{T}$.

Do this via slicing in NumPy




In [ ]:

U_r = ...

V_r = ...

S_r = ...


# Check that this decomposition equals A
np.isclose(U_r@S_r@V_r.T, A)


In [ ]:
grader.check("q1c")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 1d  ###



i). Find an orthonormal basis for $\operatorname{col}(A)$.   Give your answer as numpy matrix where the columns are the basis vectors.

ii).   Recall we are interested in the least squares solution to   $A\,\mathbf{x} = \mathbf{b}$ where  
$A = \begin{bmatrix} 1 & 0 \\ 1 & 1 \\ 1 & 2 \end{bmatrix}, \quad 
\mathbf{b} = \begin{bmatrix} -1 \\ 3 \\ 6 \end{bmatrix}.$



Let $\mathbf{\hat{b}}$ be the projection of $\mathbf{b}$ onto $\operatorname{col}(A)$.  Find $\mathbf{\hat{b}}$




In [ ]:

col_A = ...

b_hat = ...


In [ ]:
grader.check("q1d")

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 1e  ###

Answer the following questions using Markdown (and LaTeX where necessary) in the cell below:

i). What is the product $V_{r}\,V_{r}^{T}$ and why does it have this form?

ii). Show mathematically why $\hat{\mathbf{x}} = V_{r}\,\Sigma_{r}^{-1}\,U_{r}^{T}\,\mathbf{b}$  is the least-squares approximate solution to $A\mathbf{x} = \mathbf{b}$.  


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 1f  ###

The matrix  
$$A^{\dagger} = V_{r} \Sigma_{r}^{-1} U_{r}^{T}$$
is known as the **Moore–Penrose pseudoinverse** of $A$. 

When $A$ is invertible  $A^{-1} = A^{\dagger}$.

 - i).  Find $A^{\dagger}$.

 - ii).  Use your work in part(i) to solve for $\mathbf{\hat{x}}$, the least squares solution to $A\mathbf{x} = \mathbf{b}$






In [ ]:

A_pseudoinv = ...

b_hat = ...


b_hat

In [ ]:
grader.check("q1f")

This question demonstrates the power of a singular value decomposition to find a least-squares approximate solution for an equation $A\,\mathbf{x} = \mathbf{b}$. Because it immediately provides an orthonormal basis for $\operatorname{col}(A)$, something that we’ve had to construct using the Gram–Schmidt process in the past, we can easily project $\mathbf{b}$ onto $\operatorname{col}(A)$, which results in a simple expression for $\hat{\mathbf{x}}$.





If the columns of $A$ are linearly independent, then the equation $A \hat{\mathbf{x}} = \hat{\mathbf{b}}$ has a unique solution, so there is a unique least-squares approximate solution $\hat{\mathbf{x}}$. 

Otherwise, the expression above produces the solution of $A \hat{\mathbf{x}} = \hat{\mathbf{b}}$ having the shortest length.

NumPy has a built-in method to calculated the "economy" or "truncated" SVD that you calculated by hand above:

In [ ]:
U_economy, S_economy, Vt_economy = np.linalg.svd(A, full_matrices = False)

> `full_matrices = False` truncates the number of columns of U to the
> rank of A to avoid unnecessary computation.  This is sometimes called the "economy" SVD. 


In [ ]:
print("Shape of economy U:", U_economy.shape)
print("Shape of economy S:", S_economy.shape)
print("Shape of economy Vt:", Vt_economy.shape)


## SVD and Principal Component Analysis

Given a high dimensional dataset, PCA allows us to:
1. Understand the rank of the data. If $k$ principal components capture almost all of the variance, then the data is roughly rank $k$.
2. Create 2D scatterplots of the data. Such plots are a rank 2 representation of our data, and allow us to visually identify clusters of similar observations.

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 2  ###


In this question, we're going to look at a well-known data set that records some features about flowers called irises.

This data set records four measurements for each of 150 flowers so we will be working with $N=150$ points in ${\mathbf R}^4$.  The measurements, given in centimeters, are *sepal length*, *sepal width*, *petal length*, and *petal width*.  Moreover, the flowers each belong to one of three species of irises--iris setosa, iris versicolor, and iris virginica--and there are 50 of each species appearing in the data set.

Since the data lies in ${\mathbf R}^4$, we can't visualize it.  We will therefore apply Principal Component Analysis to reduce the dimension of the data, plot it in the plane, and start thinking about it.


Evaluate the following cell to load and display the dataset.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/davidaustinm/MTH205-W20/master/data/iris.data')

df

In [ ]:
# For PCA we will grab the columns with numeric data

data = df[df.columns[:4]]

data

We're going to work with the transpose of this data to be consistent with how we learned PCA in class (so that each column represents a different data point).

In [ ]:
data_PCA = data.T

data_PCA

To apply PCA, we will first need to center the data so that the mean of each feature is 0:

In [ ]:
# Compute row-wise means
row_means = data_PCA.mean(axis=1)

print (row_means)

# Subtract the mean of each row from every element in that row
A_flowers = data_PCA.sub(row_means, axis=0)


A_flowers

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 2a  ###



Find a singular value decomposition for `A_flowers` using NumPy.  (You will have to convert the S output from NumPy into a diagonal matrix of the appropriate size). 




In [ ]:


U_flowers, S_flowers, Vt_flowers = ...

Sd_flowers = ...

...


In [ ]:
grader.check("q2a")

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 2b  ###





Recall, when we learned PCA, we constructed the covariance matrix $C$ from a demeaned data matrix and saw that the eigenvalues and eigenvectors of $C$ tell us about the variance of the dataset in different directions. We referred to the eigenvectors of $C$ as principal components and found that projecting the data onto a subspace defined by the first few principal components frequently gave us a way to visualize the dataset. As we added more principal components, we retained more information about the original dataset. This feels similar to the rank $k$ approximations we have just seen, so let us explore the connection.

Suppose that we have a dataset with $N$ points, that $A$ represents the demeaned data matrix, that
$A = U \Sigma V^T$
is a singular value decomposition, and that the singular values of $A$ are denoted as $\sigma_i$.


 i ).  Use the definition of covariance and the singular value decomposition of $A$ to show mathematically why: 



$$C 
= U \left( \frac{1}{N} \Sigma \Sigma^T \right) U^T.$$


ii).  Using your answer in part (i), what are the principal components of $A$?

iii).  Using your answer in part (i), give a formula for $V_{\mathbf{u_i}}$ (the variance in the direction of the ith principal component) in terms of the singular value(s) of $A$

$$V_{\mathbf{u_i}} = $$



Answer all 3 of the questions above in the Markdown cell below


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 2c  ###





 - i ). Use the formula you found in the previous part to find the variance in each of the directions of the principal components.  Give your answer as a NumPy array (in order from largest to smallest variance)

 - ii).  What is the total variance of $A$?

 - iii.  What fraction of the total variance do the first two principal components account for?

In [ ]:

var_prin_components = ...

tot_var = ...

frac_tot_var = ...



In [ ]:
grader.check("q2c")

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 2d  ###




 - i ). Find the matrix $\Gamma$ so that $A=U\Gamma$.    Notice that a column of $\Gamma$ gives the **coefficients that express the corresponding column of $A$ as a linear combination of principal components**. 

 - ii).  What are the coefficients that express the 30th flower (indexed by 29) as a linear combination of the principal components? 



iii). After answering i) and ii) using the code cell below, answer the following question:

Which coefficient of the 30th flower is largest and why would you expect this?

**Your Answer to part (iii) Here** 

In [ ]:

gamma = ...



coeff_flower_30 = ...



In [ ]:
grader.check("q2d")

<!-- END QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 2e  ###




The $i^{th}$ row of $\Gamma$ represents the coefficients of ${\mathbf u}_i$ for each flower.  

 - i). Find the matrix $\Gamma_2$ that represents the coefficients of the dataset **projected onto the first two principal components.**

 - ii). What are the coordinates in the PCA plane of the 30th flower (which is indexed by 29)? 

 - iii). To which species will this flower belong?



In [ ]:
# Part (i) 
gamma_2 = ...



In [ ]:
# Run this cell to plot the each of the data points in the plane defined by the first two principal components.

proj = pd.DataFrame((gamma_2).T, columns=[0,1])
proj['species'] = df['species']
fig = plt.figure(figsize=(10,5))
ax = plt.gca()
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
#ax.set_aspect(1)
sns.scatterplot(x = 0, y=1, hue='species', style='species', data=proj, ax=ax)
plt.show()

In [ ]:
# Part (ii)

coord_30_flower = ...

coord_30_flower

In [ ]:
# Part (iii)

# MULTIPLE CHOICE:  
# Using the coordinates you found above and the plot shown above, what species does the 30th flower belong to?

# A:  Iris - Setosa
# B:  Iris-versicolor
# C:  Iris - virginica

#Give your answer as a string 'A' or 'B' or 'C'

part_3_answer = ...




In [ ]:
grader.check("q2e")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />



## <span style='color:Red'>   Question 2f  ###

Remember how we've gotten here:  we first demeaned the data and then projected it onto the subspace spanned by <code>u1</code> and <code>u2</code>.  

Suppose we have a flower whose coordinates in this Principal Component (PC1 and PC2) plane are $(-2.5, -0.75)$.  Estimate the four measurements (sepal length, sepal width, petal length, and petal width) for this flower.  (Hint: Think about what the Principal components and their coordinates represent). 



In [ ]:

...

sepal_length = ...
sepal_width = ...
petal_length = ...
petal_width = ...


In [ ]:
grader.check("q2f")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


### Submission Instructions

Before proceeding any further, **save this notebook.**

Then run the cell below to double check that you don't have any spaces between dollar signs and text when writing LaTeX:

In [ ]:
# Run this cell before you run the 'grader.export()' cell below.  
# It will search for LaTeX errors that will cause the LaTeX compiler to fail.  

import simple_latex_checker as slc

nb = slc.Nb_checker()
nb.run_check("lab06.ipynb")

After running the `grader.export()` cell provided below, **2 files will be created**: a zip file and pdf file.  You can download them using the links provided below OR by finding them in the same folder where this juptyer notebook resides in your JuptyerHub.

To receive credit on this assignment, **you must submit BOTH of these files
to their respective Gradescope portals:** 


* **Lab 6 Autograded**: Submit the zip file that is output by the `grader.export()` cell below to the Lab 6 Autograded assignment in Gradescope.

* **Lab 6 Manually Graded**: Submit your lab05.PDF to the Lab 6 Manually Graded assignment in Gradescope.  **It is your responsibility to fully review your PDF file before submitting and make sure that all your lines of code are visible and any LaTeX has correctly compiled and is fully viewable.**  **YOU MUST SELECT THE PAGES CORRESPONDING TO EACH QUESTION WHEN YOU UPLOAD TO GRADESCOPE.** If not, you will lose points.    

[TROUBLESHOOTING TIPS](https://docs.google.com/document/d/1ndr3Wj1PSF5qzlLMaBJznwh6QGeEXjd5TAJ6nf9EJvo/edit?usp=sharing)  If you are having any issues compiling your assignment, please read through these troubleshooting tips first, then post any questions on Piazza.  

**You are responsible for ensuring your submission follows our requirements. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

AFTER running the cell below, click on <a href='lab06.pdf' download>this link to download the PDF </a> to upload to Gradescope.  There will be a separate link that appears after running the cell below with a link to download the zip file to upload to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)